<a href="https://colab.research.google.com/github/chanhyeong00/machine_learning_study/blob/main/%08Pytorch/basic-%EB%B9%84%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

### 데이터셋 구성

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]]) # 결과값은 이차방정식(y=w_1*x^2 + W_2 * x + b)
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

### 모델 구성

In [ ]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1) # (2차 다항식, 1개의 출력)

    def forward(self, x):
        x = self.layer(x)
        return x

데이터셋이 없어서 그냥 코드로만 본다.

In [ ]:
train_dataset = CustomDataset("../datasets/non_linear.csv")
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu" # GPU 사용 가능 환경이면 'cuda'로 gpu 사용.
model = CustomModel().to(device) # 모델을 GPU로 장치 설정
criterion = nn.MSELoss().to(device) # 오차 또한 GPU로 장치 설정
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [ ]:
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device) # 텐서 모두 동일 장치로 올려줌
        y = y.to(device)

        output = model(x) # 예측
        loss = criterion(output, y) # 오차 계산

        optimizer.zero_grad() # 기울기 초기화
        loss.backward() # 역전파 수행(기울기 계산)
        optimizer.step() # 계산된 가중치를 옵티마이저에 반영

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

### 모델 평가

테스트 데이터세트나 임의값으로 모델을 확인하거나 평가할 때는 troch.no_grad 클래스를 활용한다.

troch.no_grad 클래스는 기울기 계산을 비활성화하는 클래스이다. 자동 미분 기능을 사용하지 않도록 설정해 메모리 사용량을 줄여 추론에 적합한 상태로 변경

In [ ]:
with torch.no_grad():
    model.eval() # 평가모드로 변경
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    ).to(device)
    outputs = model(inputs) # 예측(추론)
    print(outputs)

In [ ]:
torch.save(
    model,
    "../models/model.pt"
)
torch.save(
    model.state_dict(),
    "../models/model_state_dict.pt"
) # 특정 시점 모델 상태